Import Carbonara's Data Tools by running the below cell (_ctrl+enter_)

In [ ]:
import CarbonaraDataTools as CDT
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio
pio.renderers.default='notebook'
from glob import glob
import traitlets
from tkinter import Tk, filedialog

1. Enter the path to your protein (it should be inside newFitData)

In [ ]:
class SelectRunDirButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectRunDirButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Output Dir"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askdirectory()

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
        
RunDirButton = SelectRunDirButton()
RunDirButton

2. Check the progress of your run by selecting a log file from the dropdown

In [ ]:
# <-- Run this cell to view the dropdown box
working_path = RunDirButton.files[0]
logs = CDT.getLogs(working_path)
molPath = '/'.join(working_path.split('/')[:-1])+'/'

logFile_Dropdown = widgets.Dropdown(
    options = logs,
    value = logs[0],
    description = 'Select a logFile from the dropdown box ',
    disabled=False,
    layout={'width': '550px','height':'25px'}
)

# Function to display tail of DataFrame
def display_tail(log_path):
    df = CDT.LogFile2df(log_path)
    display(df.tail())

def dropdown_handler(change):
    global log_path
    log_path = change.new
    with output:
        clear_output(wait=True)  # Clear previous output
        display_tail(log_path)
        CDT.viewBestSAXSFit(molPath,log_path).show("notebook")

# Apply CSS style to the dropdown box to make it more visible
logFile_Dropdown.style = {'description_width': '250px','description_height': '25px'}

# Output widget
output = widgets.Output()
def render_initial(change=None):
        with output:
            clear_output(wait=True)
            display_tail(log_path)
            CDT.viewBestSAXSFit(molPath,log_path).show("notebook")
# Display initial output
log_path = logs[0]
logFile_Dropdown.observe(dropdown_handler,names='value')
display(logFile_Dropdown)
display(output)
render_initial()

3. Look at the current best fit conformation

In [ ]:
# <-- Run this cell to plot the current best fit conformation

CDT.plot_best_mols(molPath,log_path).show("notebook")

4. Look at the changes from start to best

In [ ]:
CDT.viewBestMolChange(molPath,log_path)

## (Warning: Not yet fully functional) 

In [ ]:
CDT.PlotBestOutputRMSD(working_path)

5. View the changes to your structure as we included more of the SAXs data.

In [ ]:
# <-- Run this cell to view the q slider

q = CDT.getqChanges(log_path)
qrange_slider = widgets.SelectionSlider(
    options=q,
    value = q[0],
    description='Max q:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '400px', 'height': '100px'}
    )

def slider_handler(change):
    global q_choice
    q_choice = change.new
    with output:
        clear_output(wait=False)
        saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q_choice)
        CDT.plotMolAndSAXS(molPath,saxs_fl,mol_fl).show("notebook")

def render_initial(change=None):
    with output:
        clear_output(wait=True)
        CDT.plotMolAndSAXS(molPath,saxs_fl,mol_fl).show("notebook")
        
        
display(qrange_slider)
saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q[0])
# Output widget
output = widgets.Output()
qrange_slider.observe(slider_handler,names='value')
display(output)
render_initial()